# Question and Answering Using Embeddings

Embeddings are a way of representing words or phrases as numerical vectors that can be used as input to machine learning algorithms. Word embeddings map words to dense vectors of real numbers, with the property that words that are semantically similar are mapped to vectors that are close together in a high-dimensional space.

In this notebook we will demonstrate how we can use embeddings to return better output responses using embeddings based search


### Pre-requisites

Firstly we need to install relevant packages to allow us to execute the code. For this tutorial we will need to install openai which is a package that allows you to use the OpenAI endpoints from within your notebook.

In [2]:
!pip install openai transformers --upgrade
#!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html


### Load the python libraries

Then we need to load all of the relevant libraries in order to execute this tutorial script

In [17]:
import openai
import numpy as np
import pandas as pd
import transformers
from openai import AzureOpenAI
from transformers import AutoTokenizer

c:\Users\mahesh.varia\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
%pip install transformers

  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
  Using cached filelock-3.13.4-py3-none-any.whl.metadata (2.8 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.0 kB 325.1 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 337.6 kB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
Using cached transformers-4.40.1-py3-none-any.whl (9.0 MB)
Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)
   ---------------------------------------- 0.0/138.7 kB ? eta -:--:--
   ---------------------------------------- 138.7/138.7 kB 4.1 MB/s eta 0:00:00
   ----------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [41]:
tokens_model = "gpt-35-turbo"
embeddings_model = "genai-training-text-embedding-ada-002"
chatgpt35_model_name = "genai-training-gpt-35-turbo"

### Set your OpenAI API key

Now, we set our OpenAI API key. You can find yours here https://platform.openai.com/account/api-keys

In [35]:
chat_client = AzureOpenAI(
    api_key="e21f323f54f14baab3e5931d37908b9b",
    api_version="2024-02-15-preview",
    azure_endpoint="https://oai-genai-training-2310.openai.azure.com/"
)

In [37]:
def get_embedding(text, model):
    return chat_client.embeddings.create(input = [text], model=model).data[0].embedding

In [38]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In this tutorial we will be using the ChatCompletion.create() endpoint that will allow us to query the **GPT-3.5-turbo** and **GPT-4** models - these are the most advanced and up-to-date models OpenAI have released for us to use

Now you are ready to start with the rest of the tutorial

## Introduction

Whilst GPT models are incredibly impressive at answering with coherent responses to questions, but only on data that it can remember during training. There are some slight limitations when adopting these models for use case specific tasks:

**Use case specific terminology:** A client may want to ask the model questions about company policies. Because these policies are not publicly available information the model will not have seen it during training and therefore will not be able to respond accurately

**Real time Intelligence:** The models have only been trained on data up to a certain date (for GPT-3: January 2021, for GPT-3.5-turbo and GPT-4 this was September 2021)

**Information from previous conversations:** If you would like to use GPT as a question-answer bot where there may be follow up questions from the previous response

The above considerations are crucial to be solved in order to start tailoring these models to a client use case. This is were **embeddings** step in, embeddings help the model to learn use case specific information by inserting additional information into the input message.

By inputting additional information into the model input the model will answer the question using the information provided in the prompt. We are basically handing both the question and answer to the model and asking the model to process this information and output a coherent answer.

Use cases that would benefit from this technique are ones that involve a heavy amount of manual searching. i.e. You want to find out what your sick pay policy at work but the policy document is 10 pages long. We can add the policy alongisde the question 'What are my sick pay conditions?' into the prompt and allow the model the search the document and output the required information.

Large Language Models are much faster at processing this information, massively reducing the amount of human processing times

### Example - The UK Prime Minister

In this example we will show how we can help the models to answer questions to events that have happened post training data date:

In [10]:
question = 'Who is the Prime Minister of the United Kingdom?'

response = chat_client.chat.completions.create(
        model=chatgpt35_model_name,
        messages=[
        {'role': 'system', 'content': 'You answer questions about UK politics'},
        {'role': 'user', 'content': question},
        ],
    )
print(response.choices[0].message.content)

As of October 2021, the Prime Minister of the United Kingdom is Boris Johnson. He has been in office since July 24, 2019, and is the leader of the Conservative Party.


The model has provided an answer that is incorrect - Boris Johnson was a priminister for the United Kingdom but there has been 2 priministers since he was in power. As of October 2022 Rishi Sunak has been the Priminister for the UK.

Even though GPT-3.5-turbo is one of the leading LLMs in the market, it was not able to provide the correct answer. This is because the model has been trained on data up until September 2021, so it will not know about this change of priminster as that happened post training date.

### Providing the model with use case specific information to learn from

We can help the model to learn about these recent events by inserting knowledge into our input message. Below I have pasted the list of all UK priministers from Wikipedia for the model to learn from

In [11]:
list_of_uk_priministers = """
The prime minister of the United Kingdom is the principal minister of the crown of His Majesty's Government, and the head of the British Cabinet. There is no specific date for when the office of prime minister first appeared, as the role was not created but rather evolved over a period of time through a merger of duties.[1] The term was regularly, if informally, used of Robert Walpole by the 1730s.[2] It was used in the House of Commons as early as 1805,[3] and it was certainly in parliamentary use by the 1880s.[4] In 1905, the post of prime minister was officially given recognition in the order of precedence.[5]
Since 1721

Prime ministers
photograph	Ramsay MacDonald[79]
MP for Seaham
(1866–1937)	5 June
1929	7 June
1935	6 years, 3 days	(1929)
First Lord of the Treasury
Leader of the House of Commons
Labour	MacDonald II
(—)	National Labour	National I
(Nat.Lab–​Con–​others)
1931	National II
​	photograph	Stanley Baldwin[80]
MP for Bewdley
(1867–1947)	7 June
1935	28 May
1937	1 year, 356 days	1935
First Lord of the Treasury
Leader of the House of Commons
Conservative	National III
Edward VIII
Edward VIII
r. 1936
George VI
George VI
r. 1936–1952
photograph	Neville Chamberlain[81]
MP for Birmingham Edgbaston
(1869–1940)	28 May
1937	10 May
1940	2 years, 349 days	—
First Lord of the Treasury
Leader of the House of Commons
National IV
Chamberlain War
photograph	Winston Churchill[82]
MP for Epping
(1874–1965)	10 May
1940	26 July
1945	5 years, 78 days	—
First Lord of the Treasury
Leader of the House of Commons (1940–1942)
Minister of Defence
Churchill War
Churchill Caretaker
(Con–​Nat.Lib)
photograph	Clement Attlee[83]
MP for Limehouse
(1883–1967)	26 July
1945	26 October
1951	6 years, 93 days	1945
First Lord of the Treasury
Minister of Defence (1945–1946)
Labour	Attlee I
1950	Attlee II
photograph	Winston Churchill[84]
MP for Woodford
(1874–1965)	26 October
1951	5 April
1955	3 years, 162 days	1951
First Lord of the Treasury
Minister of Defence (1951–1952)
Conservative	Churchill III
Elizabeth II
Elizabeth II
r. 1952–2022
photograph	Anthony Eden[85]
MP for Warwick and Leamington
(1897–1977)	6 April
1955	9 January
1957	1 year, 279 days	1955
First Lord of the Treasury
Eden
photograph	Harold Macmillan[86]
MP for Bromley
(1894–1986)	10 January
1957	18 October
1963	6 years, 282 days	—
First Lord of the Treasury
Macmillan I
1959	Macmillan II
photograph	Alec Douglas-Home[87][f]
MP for Kinross and Western Perthshire
(1903–1995)	18 October
1963	16 October
1964	365 days	—
First Lord of the Treasury
Conservative
(Scot.U.)	Douglas-Home
photograph	Harold Wilson[88]
MP for Huyton
(1916–1995)	16 October
1964	19 June
1970	5 years, 247 days	1964
First Lord of the Treasury
Minister for the Civil Service (1968–1970)
Labour	Wilson I
1966	Wilson II
photograph	Edward Heath[89]
MP for Bexley
(1916–2005)	19 June
1970	4 March
1974	3 years, 259 days	1970
First Lord of the Treasury
Minister for the Civil Service
Conservative	Heath
photograph	Harold Wilson[88]
MP for Huyton
(1916–1995)	4 March
1974	5 April
1976	2 years, 33 days	(Feb.1974)
First Lord of the Treasury
Minister for the Civil Service
Labour	Wilson III
Oct.1974	Wilson IV
P.M. Callaghan	James Callaghan[90]
MP for Cardiff South East
(1912–2005)	5 April
1976	4 May
1979	3 years, 30 days	—
First Lord of the Treasury
Minister for the Civil Service
Callaghan
photograph	Margaret Thatcher[91]
MP for Finchley
(1925–2013)
Premiership	4 May
1979	28 November
1990	11 years, 209 days	1979
First Lord of the Treasury
Minister for the Civil Service
Conservative	Thatcher I
1983	Thatcher II
1987	Thatcher III
John Major 1996.jpg	John Major[92]
MP for Huntingdon
(born 1943)
Premiership	28 November
1990	2 May
1997	6 years, 156 days	—
First Lord of the Treasury
Minister for the Civil Service
Major I
1992	Major II
photograph	Tony Blair[93]
MP for Sedgefield
(born 1953)
Premiership	2 May
1997	27 June
2007	10 years, 57 days	1997
First Lord of the Treasury
Minister for the Civil Service
Labour	Blair I
2001	Blair II
2005	Blair III
photograph	Gordon Brown[94]
MP for Kirkcaldy and Cowdenbeath
(born 1951)
Premiership	27 June
2007	11 May
2010	2 years, 319 days	—
First Lord of the Treasury
Minister for the Civil Service
Brown
photograph	David Cameron[95]
MP for Witney
(born 1966)
Premiership	11 May
2010	13 July
2016	6 years, 64 days	(2010)
First Lord of the Treasury
Minister for the Civil Service
Conservative	Cameron–Clegg
(Con–​Lib.Dem)
2015	Cameron II
photograph	Theresa May[96]
MP for Maidenhead
(born 1956)
Premiership	13 July
2016	24 July
2019	3 years, 12 days	—
First Lord of the Treasury
Minister for the Civil Service
May I
(2017)	May II
photograph	Boris Johnson[97]
MP for Uxbridge and South Ruislip
(born 1964)
Premiership	24 July
2019	6 September
2022	3 years, 45 days	(—)
First Lord of the Treasury
Minister for the Civil Service
Minister for the Union
Johnson I
2019	Johnson II
photograph	Liz Truss[98]
MP for South West Norfolk
(born 1975)
Premiership	6 September
2022	25 October
2022	50 days	—
First Lord of the Treasury
Minister for the Civil Service
Minister for the Union
Truss
Charles III
Prince Charles in Aotearoa (cropped).jpg
r. 2022–present
photograph
Rishi Sunak[99]
MP for Richmond (Yorks)
(born 1980)
Premiership	25 October
2022Incumbent	174 days	—
First Lord of the Treasury
Minister for the Civil Service
Minister for the Union
Sunak
"""

Now lets try and ask the same question but add this additional information into our prompt

In [12]:
query = f"""Use the information provided in the wikipedia extract below to answer questions on UK priministers,
If the answer cannot be found, write "I don't know."

Additional information:
\"\"\"
{list_of_uk_priministers}
\"\"\"

Question: Who is the Priminister of the United Kingdom?"""
response = chat_client.chat.completions.create(
        model=chatgpt35_model_name,
        messages=[
        {'role': 'system', 'content': 'You answer questions about UK politics.'},
        {'role': 'user', 'content': query},
    ],
    )
print(response.choices[0].message.content)

The current Prime Minister of the United Kingdom is Rishi Sunak.


The model has now been able to provide the correct answer to our question! This shows that given some use case specific information - the model can now be tailored for that use case, this means we can start applying them to a wide variety of use cases **given we have the required data**

For the example above - I had to search on the internet and find information on the question I was looking for before pasting it into my prompt. This can be rather time consuming. One suggestion could be to copy and paste the whole of Wikipedia into our prompt as our additional knowldege. Whilst this would be a dream scenario it is not possible as there is a maximum limit to the amount of text these models can read at once:

* **GPT-3:** 4,096 Max token length (~5 pages of text)
* **GPT-3.5-turbo:** 4,096 Max token length (~5 pages of text)
* **GPT-4:** 8,192 Max token length (~10 pages of text)


So we need to find a way of adding additional knowledge into our input message by finding the most relevant information from our use case specific dataset and selecting only the relevant chunks of text to be added in to the prompt. **Now this is where we use embeddings**

### The Process

In order to extract only the relevant information from our dataset to be used as additional knowledge for our model we follow these steps:

    1. Prepare our use case specific dataset
         Collect: We'll download a few hundred Wikipedia articles about the 2022 Olympics
         Chunk: Documents are split into short, mostly self-contained sections to be embedded
         Embed: Each section is embedded with the OpenAI API
         Store: Embeddings are saved (for large datasets, use a vector database)
         
    2. Search for relevant information
         Given a user question, generate an embedding for the query from the OpenAI API
         Using the embeddings, rank the text sections by relevance to the query
         
    3. Ask (once per query)
         Insert the question and the most relevant sections into a message to GPT
         Return GPT's answer
         
### Step 1: Prepare our use case specific dataset
In this session folder we have already provided the dataset we are wanting to query against. We have scraped the Accenture US Careers page and have saved the extracted text alongside the file name into a csv

In [11]:
#we first upload the dataset
import pandas as pd
raw_data_df = pd.read_excel('C:\\GenAI_Training\\Embeddings\\accenture_careers_scraper.xlsx',index_col=0, engine='openpyxl')

In [10]:
%pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


Let's have a look at the first 10 outputs and count how many website URLs we have scraped Accenture Career from

In [12]:
raw_data_df.columns = ['fname', 'text']
raw_data_df.head()
print("The total number of files is:",len(raw_data_df))

The total number of files is: 84


In [13]:
raw_data_df.head()

fname  \
0              us en careers local military veterans   
1                        us en careers local levelup   
2   us en careers explore careers area of interes...   
3   us en careers life at accenture work environment   
4   us en careers life at accenture people profil...   

                                                text  
0   us en careers local military veterans.   Jobs...  
1   us en careers local levelup.   Level Up      ...  
2   us en careers explore careers area of interes...  
3   us en careers life at accenture work environm...  
4   us en careers life at accenture people profil...

Now we count the number of tokens for each webpage text, we use the GPT2TokennizerFast package for this

In [18]:

tokenizer = AutoTokenizer.from_pretrained("gpt2")
raw_data_df['n_tokens'] = raw_data_df.text.apply(lambda x: len(tokenizer.encode(x)))

c:\Users\mahesh.varia\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mahesh.varia\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Token indices sequence length is longer than the specified maximum sequence length for this 

This will create a column that provides a count for the number of tokens

In [26]:
raw_data_df.head()

fname  \
0              us en careers local military veterans   
1                        us en careers local levelup   
2   us en careers explore careers area of interes...   
3   us en careers life at accenture work environment   
4   us en careers life at accenture people profil...   

                                                text  n_tokens  
0   us en careers local military veterans.   Jobs...      2746  
1   us en careers local levelup.   Level Up      ...      2820  
2   us en careers explore careers area of interes...      3367  
3   us en careers life at accenture work environm...      2535  
4   us en careers life at accenture people profil...      2891

As you can see from the 'n_tokens' column, our webpages have quite a lot of text! Given the GPT-3.5-turbo model has a 4,096 token limit we would only be able to realistically include one webpage in our input message. This will reduce the capabilities of our use case solution as it limits the range of questions we can ask about careers.

The next step from here is to split our files into chunks of text that can easily fit into our input prompt.

### Splitting the text into chunks

The function below takes in a text and splits it into smaller chunks of text based on a maximum number of tokens defined by the variable "max_tokens".

It then loops through each row in a dataframe and checks if the text in that row is longer than the maximum number of tokens. If it is, it calls the "split_into_many" function to split the text into smaller chunks and appends each processed text chunk and its corresponding title to a list called "processed_text".

If the text is not longer than the maximum number of tokens, it simply appends the original text and its corresponding title to the "processed_text" list.

In [27]:
#we will split the text into chunks of 500 tokens
max_tokens = 500

# Define a function that splits long job postings into smaller chunks of text
def split_into_many(text, max_tokens=max_tokens):
    sentences = text.split('.')
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]

    chunks = []
    tokens_so_far = 0
    chunk = []
    for s, t in zip(sentences, n_tokens):
        if tokens_so_far + t > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0
        if t > max_tokens:
            continue
        chunk.append(s)
        tokens_so_far += t + 1
    return chunks

# Initialize an empty list to store the processed text and corresponding titles
processed_text = []

# Loop through each row in the dataframe
for row in raw_data_df.iterrows():
    if row[1]['text'] is None:
        continue
    if row[1]['n_tokens'] > max_tokens:
        # Split long job postings into smaller chunks of text
        chunks = split_into_many(row[1]['text'])
        for chunk in chunks:
            # Append each processed text chunk and its corresponding title to the list
            processed_text.append({'fname': row[1]['fname'], 'text': chunk})
    else:
        # Append the original text and its corresponding title to the list
        processed_text.append({'fname': row[1]['fname'], 'text': row[1]['text']})

We now convert our list into a dataframe with each chunk being added per row

In [30]:
processed_df = pd.DataFrame(processed_text)
processed_df

fname  \
0                us en careers local military veterans   
1                us en careers local military veterans   
2                us en careers local military veterans   
3                us en careers local military veterans   
4                          us en careers local levelup   
..                                                 ...   
346            us en careers local return work program   
347   us en careers explore careers area of interes...   
348   us en careers explore careers area of interes...   
349   us en careers explore careers area of interes...   
350   us en careers explore careers area of interes...   

                                                  text  
0     us en careers local military veterans.    Job...  
1    S.  IN THE U. S.  About Accenture About Accent...  
2     And together, we are delivering meaningful ch...  
3     Perry \t\t\t\t\t\t \t\t\t\t\t\t\t\tMILITARY R...  
4     us en careers local levelup.    Level Up     ...  
..                                                 ...  
346      Receive on-the-job shadow training and lea...  
347   us en careers explore careers area of interes...  
348  S.  IN THE U. S.  About Accenture About Accent...  
349   A gateway to all aspects of our Alumni progra...  
350  Â  The Company will not discharge or in any ot...  

[351 rows x 2 columns]

We can run the code below to check and see the max tokens for each of the rows to check there are no more than 500 tokens in length

In [31]:
check_length = processed_df
tokenizer = AutoTokenizer.from_pretrained("gpt2")
check_length['n_tokens'] = check_length.text.apply(lambda x: len(tokenizer.encode(x)))

max_n_tokens = check_length['n_tokens'].max()
print("Maximum value of n_tokens:", max_n_tokens)

Maximum value of n_tokens: 501


Now that we have split our data into chunks of 500 we want to create a unique ID so we can trace back the exact chunk that has been used in the prompt. This is an important feature to have as it helps you identify and validate that your code is selecting the relevant information to be added into your prompt

In [32]:
def apply_unique_id(df):
    df['ID'] = [ i  for i in range(1, df.shape[0] + 1)]
    df['Source ID'] = df['fname'].astype(str) + '_' + df['ID'].astype(str)
    return df

In [33]:
embeddings_df = apply_unique_id(processed_df)
embeddings_df

fname  \
0                us en careers local military veterans   
1                us en careers local military veterans   
2                us en careers local military veterans   
3                us en careers local military veterans   
4                          us en careers local levelup   
..                                                 ...   
346            us en careers local return work program   
347   us en careers explore careers area of interes...   
348   us en careers explore careers area of interes...   
349   us en careers explore careers area of interes...   
350   us en careers explore careers area of interes...   

                                                  text  n_tokens   ID  \
0     us en careers local military veterans.    Job...       126    1   
1    S.  IN THE U. S.  About Accenture About Accent...       500    2   
2     And together, we are delivering meaningful ch...       497    3   
3     Perry \t\t\t\t\t\t \t\t\t\t\t\t\t\tMILITARY R...       241    4   
4     us en careers local levelup.    Level Up     ...       119    5   
..                                                 ...       ...  ...   
346      Receive on-the-job shadow training and lea...       333  347   
347   us en careers explore careers area of interes...       123  348   
348  S.  IN THE U. S.  About Accenture About Accent...       470  349   
349   A gateway to all aspects of our Alumni progra...       469  350   
350  Â  The Company will not discharge or in any ot...       410  351   

                                             Source ID  
0              us en careers local military veterans_1  
1              us en careers local military veterans_2  
2              us en careers local military veterans_3  
3              us en careers local military veterans_4  
4                        us en careers local levelup_5  
..                                                 ...  
346        us en careers local return work program_347  
347   us en careers explore careers area of interes...  
348   us en careers explore careers area of interes...  
349   us en careers explore careers area of interes...  
350   us en careers explore careers area of interes...  

[351 rows x 5 columns]

### Creating the embeddings for our dataset

OpenAI provide an embeddings model called **text-embedding-ada-002** which converts our text string into embeddings

In [44]:
embedding_type = 'text-embedding-ada-002'

#NOTE: The commented out code only needs to be run once for each dataset.

embeddings_df['embeddings'] = embeddings_df.text.apply(lambda x: get_embedding(x, embeddings_model))

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2024-02-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}

In [64]:
embeddings_df

fname  \
0                us en careers local military veterans   
1                us en careers local military veterans   
2                us en careers local military veterans   
3                us en careers local military veterans   
4                          us en careers local levelup   
..                                                 ...   
346            us en careers local return work program   
347   us en careers explore careers area of interes...   
348   us en careers explore careers area of interes...   
349   us en careers explore careers area of interes...   
350   us en careers explore careers area of interes...   

                                                  text  n_tokens   ID  \
0     us en careers local military veterans.    Job...       126    1   
1    S.  IN THE U. S.  About Accenture About Accent...       500    2   
2     And together, we are delivering meaningful ch...       497    3   
3     Perry \t\t\t\t\t\t \t\t\t\t\t\t\t\tMILITARY R...       241    4   
4     us en careers local levelup.    Level Up     ...       119    5   
..                                                 ...       ...  ...   
346      Receive on-the-job shadow training and lea...       333  347   
347   us en careers explore careers area of interes...       123  348   
348  S.  IN THE U. S.  About Accenture About Accent...       470  349   
349   A gateway to all aspects of our Alumni progra...       469  350   
350  Â  The Company will not discharge or in any ot...       410  351   

                                             Source ID  
0              us en careers local military veterans_1  
1              us en careers local military veterans_2  
2              us en careers local military veterans_3  
3              us en careers local military veterans_4  
4                        us en careers local levelup_5  
..                                                 ...  
346        us en careers local return work program_347  
347   us en careers explore careers area of interes...  
348   us en careers explore careers area of interes...  
349   us en careers explore careers area of interes...  
350   us en careers explore careers area of interes...  

[351 rows x 5 columns]

Now that we have converted our dataset into embeddings we save this down as a csv so we can search through it for different question inputs

In [65]:
#embeddings_df.to_csv('C:\\GenAI_Training\\Embeddings\\acn_careers_embeddings.csv')
embeddings_df = pd.read_csv('C:\\GenAI_Training\\Embeddings\\acn_careers_embeddings.csv',index_col=0)

### Step 2: Search for relevant information
Given a user question, generate an embedding for the query from the OpenAI API. Using the embeddings, rank the text sections by relevance to the query

This function takes in our question and embeddings dataframe, and returns a list of contexts sorted by the similarity to the question, along with the sorted dataframe and the embeddings for the question.

In [66]:
def create_context(question, df):
    """
    Create a context for a question by finding the most similar context from dataframe
    """
    q_embeddings = get_embedding(question,embeddings_model)

    df['distances'] = df['embeddings'].apply(lambda x: cosine_similarity(q_embeddings, x))

    returns = []
    cur_len = 0
    for i, row in df.sort_values('distances', ascending=False).iterrows():
        returns.append('Source ID: ' + row["Source ID"] + ' - ' + row["text"])
    return returns, df.sort_values('distances', ascending=False)

In [67]:
create_context('what kind of industry does Accenture work in?', embeddings_df)

KeyError: 'embeddings'

If we have a look at the sorted dataframe we can see that **Source ID:  us en careers explore careers area of interest oracle careers_99** contains the most relevant information for our question. We can identify this by looking at the 'distance' column that tells indicates the cosine similarity between two strings. The smaller the values the more similar the two strings are. Here the distance was 0.143099 between our question and the text found at Source ID:  us en careers explore careers area of interest oracle careers_99

In [26]:
display(embeddings_df.sort_values('distances', ascending=False))


fname  \
212   us en careers explore careers area of interes...   
227   us en careers explore careers area of interes...   
253   us en careers life at accenture people profil...   
231   us en careers explore careers area of interes...   
114   us en careers explore careers area of interes...   
..                                                 ...   
84          us en careers remove confirmation register   
270   us en careers life at accenture labor conditi...   
268   us en careers life at accenture labor conditi...   
184                us en careers registration register   
79                 us en careers profiles edit profile   

                                                  text  n_tokens   ID  \
212  S.  IN THE U. S.  About Accenture About Accent...       464  213   
227     Partner with the biggest clients.  Gain unr...       475  228   
253  S.  IN THE U. S.  About Accenture About Accent...       485  254   
231  S.  IN THE U. S.  About Accenture About Accent...       483  232   
114  S.  IN THE U. S.  About Accenture About Accent...       497  115   
..                                                 ...       ...  ...   
84    us en careers remove confirmation register.  ...       115   85   
270   Paul_Minnesota I-200-23082-871771_CSSSDA_Durh...       133  271   
268   Paul_Minnesota I-200-23094-900970_SFC_Nashvil...        95  269   
184   us en careers registration register.  DS Port...        28  185   
79    us en careers profiles edit profile.  DS Port...        29   80   

                                             Source ID  \
212   us en careers explore careers area of interes...   
227   us en careers explore careers area of interes...   
253   us en careers life at accenture people profil...   
231   us en careers explore careers area of interes...   
114   us en careers explore careers area of interes...   
..                                                 ...   
84       us en careers remove confirmation register_85   
270   us en careers life at accenture labor conditi...   
268   us en careers life at accenture labor conditi...   
184            us en careers registration register_185   
79              us en careers profiles edit profile_80   

                                            embeddings  distances  
212  [-0.002995617687702179, -0.015972143039107323,...   0.855151  
227  [-0.0036204352509230375, -0.02909732423722744,...   0.851274  
253  [-0.009385167621076107, -0.016817351803183556,...   0.849479  
231  [-0.005147191695868969, -0.018620330840349197,...   0.848284  
114  [-0.00895965751260519, -0.018025899305939674, ...   0.847409  
..                                                 ...        ...  
84   [-0.004466082900762558, -0.015667114406824112,...   0.724582  
270  [0.005667275749146938, -0.002802650211378932, ...   0.700070  
268  [7.534134783782065e-05, -0.005357979331165552,...   0.698731  
184  [-0.014241271652281284, -0.012807722203433514,...   0.698518  
79   [-0.013995518907904625, 0.002954683732241392, ...   0.696058  

[342 rows x 7 columns]

### Step 3: Add additional context into our prompt
Now we are at the final step, we have identified the most relevant chunk of text from our dataset using the cosine similarity, now we need to append this chunk into our input request so the model can generate a response from it

The following function function takes in our embeddings dataframe, an initial prompt, a question, and a context length (which is set to 5 by default).

The function then creates a list of contexts related to the input question by calling another function called "create_context". It then joins the first "context_length" number of contexts with a separator, and uses the resulting string as a prompt to generate a response using the OpenAI GPT-3 language model.

Finally, the function returns the generated response and the original dataframe containing the contexts.

In [27]:
def answer_question(df,initial_prompt, question, context_length = 5):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    contexts,contexts_df = create_context(question, df)
    contexts = '\n\n###\n\n'.join(contexts[0:context_length])

    prompt = '\n\nContext: ' + contexts +  ' \n\n---\n\nQuestion:' + question  + '\n\nAnswer:'

    response = chat_client.chat.completions.create(
        messages=[
            {'role': 'system', 'content': initial_prompt},
            {'role': 'user', 'content': prompt},
        ],
        model=chatgpt35_model_name,
    )

    return response.choices[0].message.content, contexts_df

Now if we ask our question regarding Accenture careers, it appears the model has now learn't everything there is to know about Accenture Careers in the US

In [28]:
question = 'what kind of industry does Accenture work in?'
initial_prompt = "Answer questions about the careers website"

response ,context_df= answer_question(embeddings_df, initial_prompt, question)
display(response)

'Accenture works in a variety of industries, including digital engineering and manufacturing, consulting, strategy, and operations. They partner with clients across different sectors such as technology, finance, healthcare, consumer goods, and many others, to provide a range of services, including digital transformation, management consulting, technology consulting, and operations improvement.'

In [29]:
display(response)

'Accenture works in a variety of industries, including digital engineering and manufacturing, consulting, strategy, and operations. They partner with clients across different sectors such as technology, finance, healthcare, consumer goods, and many others, to provide a range of services, including digital transformation, management consulting, technology consulting, and operations improvement.'

Let's try another question

In [31]:
job_question = 'what entry level jobs do Accenture have currently available?'

character_prompt = "Answer questions about the careers website, respond with 300 characters or less"

response ,context_df= answer_question(embeddings_df, character_prompt, job_question)
display(response)

'Currently, there are no entry-level jobs available with Accenture in the United States. You can sign up for job alerts to be notified when a matching job becomes available.'

## Exercise

Now that we have our embeddings dataset, try an pressure test it to see what questions it struggles on, if you have identified weak areas - try out some different prompt engineering techniques to see if that improves the answer